# Using SHAP values, identify a hospital that has a neutral contribution to the patient outcome
Use the model trained in notebook 010 (a multiclass classification model to give the probability that each patient is in each discharge disability class).\
Use the SHAP values as calculated from that model in notebook 011.

### Plain English summary

Future notebooks use a neutral hospital. Use this notebook to identify which hosptial has the most zero SHAP value for those patients that attend the hospital.

### Model and data
Model: XGBoost classifier (multiclass classification) [from notebook 010]\
Target feature: Discharge disability\
Input features: All the relevant features in SSNAP\
Kfold split: 5 kfold split\
[SHAP values from notebook 011]

### Aims
Identify a hospital that has a neutral contribution to the patient outcome

### Observations
No hospital has a zero SHAP for each of the mRS classes.\
Calculating the maximum of the absolute mean shap for each hospital - take the hospital with the minimum result (this is 0.136118 for Royal Lancaster Infirmary).\
Royal Lancaster Infirmary has the most consistent values close to zero.

#### Further work


#### Resources


## Import libraries

In [8]:
# Turn warnings off to keep notebook tidy
import warnings
warnings.filterwarnings("ignore")

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import auc

from dataclasses import dataclass

from sklearn.metrics import roc_auc_score

import pickle
import shap

from os.path import exists

import math

import time
import datetime

Report the time duration to run notebook

In [9]:
start_time = time.time()

Choose number of kfolds (either =1 for the first kfold, or =5 for the full 5 kfold split)

In [10]:
n_kfold = 5

## Set up paths and filenames

Use os.path.join() to create filenames. So define folders without trailing forward slash, and include all characters in file names.

In [11]:
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''
    image_save_path: str = './saved_images'
    model_save_path: str = './saved_models'
    data_save_path: str = './saved_data'
    data_read_path: str = '../data_processing/output/kfold_5fold'
    model_text: str = f'xgb_all_features_{n_kfold}fold'
    notebook: str = '013a_'

paths = Paths()

Create output folders if needed

In [12]:
path = paths.image_save_path
if not os.path.exists(path):
    os.makedirs(path)
        
path = paths.model_save_path
if not os.path.exists(path):
    os.makedirs(path)

path = paths.data_save_path
if not os.path.exists(path):
    os.makedirs(path)

Define variables

In [13]:
n_classes = 7

Load results (from notebook 013) to identify which hospitals have the most zero SHAP value for all of the output classes. 

This information is used in other notebooks.

In [14]:
# Initalise list. Store a list for each class of the hospitals that have a zero
# mean and std SHAP
list_hospital_zero_mean_std = []

for c in range(n_classes):

    filename = os.path.join(paths.data_save_path, ('013_' + 
                            paths.model_text + 
                            f'_meanabs_std_shap_per_hospital_for_mRS{c}'
                            f'.csv'))
    
    hospital_data = pd.read_csv(filename)

    filtered_values = np.where((hospital_data["mean_abs_shap"]==0) & 
                               (hospital_data["std_shap"]==0))
    list_hospital_zero_mean_std.append(
            list(hospital_data["stroke_team"].loc[filtered_values].values))

Return the number of times each hospital has zero SHAP mean and std deviation for the classes, for the first kfold

In [15]:
from collections import Counter
flatten = [item for row in list_hospital_zero_mean_std for item in row]
Counter(flatten).most_common(20)

[("Queen's Medical Centre - Nottingham", 3),
 ('Furness General Hospital', 2),
 ('Norfolk and Norwich University Hospital', 2),
 ("St Mary's Hospital Newport", 2),
 ('St Richards Hospital', 2),
 ('University Hospitals Dorset Stroke Service', 2),
 ('Bronglais Hospital', 1),
 ('Countess of Chester Hospital', 1),
 ('Hereford County Hospital', 1),
 ('Prince Charles Hospital', 1),
 ('Weston General Hospital', 1),
 ('Bradford and Airedale SU', 1),
 ('Broomfield Hospital', 1),
 ('James Paget Hospital', 1),
 ('Royal United Hospital Bath', 1),
 ('University Hospitals Bristol Inpatient Team', 1),
 ('Luton and Dunstable Hospital', 1),
 ('Royal Lancaster Infirmary', 1),
 ('Southend Hospital', 1),
 ('Doncaster Royal Infirmary', 1)]

No hospital has a zero SHAP for all classes. 

Instead, for each hospital, take the maximum of the mean of the absolute SHAPS, from the 6 target classes.

Then return the hospitals ranked with the smallest maximum mean of the absolute SHAP values (across the classes).

Select the hospital that has the smallest value.

In [17]:
# Initialise dataframes
hospital_shap_mean_all_classes=pd.DataFrame()
hospital_shap_std_all_classes=pd.DataFrame()

# For each class
for c in range(n_classes):
    # Get hosptial SHAP values (mean absolute and standard deviation for all
    # patients that attend the hospital)
    filename = os.path.join(paths.data_save_path, ('013_' + 
                            paths.model_text + 
                            f'_meanabs_std_shap_per_hospital_for_mRS{c}'
                            f'.csv'))
    
    hospital_data = pd.read_csv(filename)

    hospital_shap_mean_all_classes[f"mean_abs_shap_class{c}"] = (
                                                hospital_data["mean_abs_shap"])
    hospital_shap_std_all_classes[f"std_shap_class{c}"] = (
                                                hospital_data["std_shap"])
hospital_shap_mean_all_classes.set_index(hospital_data["stroke_team"].values, 
                                         inplace=True)
hospital_shap_std_all_classes.set_index(hospital_data["stroke_team"].values, 
                                        inplace=True)

df_results = pd.DataFrame()
df_results["max_of_the_absolute_mean_shap"] = (
                        np.absolute(hospital_shap_mean_all_classes).max(axis=1))
df_results.set_index(hospital_shap_std_all_classes.index, inplace=True)
df_results.sort_values(by=["max_of_the_absolute_mean_shap"], key=abs, 
                       inplace=True)
df_results

,max_of_the_absolute_mean_shap
Royal Lancaster Infirmary,0.136118
St Richards Hospital,0.159492
Stepping Hill Hospital,0.160162
Wycombe General Hospital,0.197811
Norfolk and Norwich University Hospital,0.197916
...,...
Royal Stoke University Hospital,1.441757
Royal London Hospital HASU,1.527460
Sandwell District Hospital,1.566673
Princess Royal University Hospital HASU,1.728447


View the individual SHAP values across the seven target classes for this chosen hospital

In [18]:
hospital_shap_mean_all_classes.loc["Royal Lancaster Infirmary"]

mean_abs_shap_class0    0.031250
mean_abs_shap_class1    0.136118
mean_abs_shap_class2    0.047157
mean_abs_shap_class3    0.000000
mean_abs_shap_class4    0.035217
mean_abs_shap_class5    0.067202
mean_abs_shap_class6    0.016733
Name: Royal Lancaster Infirmary, dtype: float64

Duration to run notebook

In [19]:
str(datetime.timedelta(seconds=(time.time()-start_time)))

'0:00:28.748199'